In [ ]:

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

def generate_data():
    platforms = ['web', 'ios', 'android']
    events = ['login', 'logout', 'play_video', 'like', 'comment']
    users = [f'user_{i}' for i in range(1, 101)]
    videos = [f'video_{i}' for i in range(1, 51)]

    data = []
    for i in range(2000):
        event = random.choice(events)
        record = {
            'id': f'ev_{i}',
            'user_id': random.choice(users),
            'timestamp': datetime.now() - timedelta(days=random.randint(0, 90)),
            'platform': random.choice(platforms),
            'type': event,
            'video_id': random.choice(videos) if event == 'play_video' else None,
            'time_sec': random.randint(30, 600) if event == 'play_video' else None,
            'duration_sec': random.randint(60, 1200) if event == 'play_video' else None
        }
        data.append(record)
    return pd.DataFrame(data)

df = generate_data()
print(df.head())

def calculate_score(row):
    if row['type'] == 'comment': return 5
    elif row['type'] == 'like': return 3
    elif row['type'] == 'play_video':
        if pd.notna(row['time_sec']) and row['duration_sec'] > 0:
            return min(3, max(1, int(3 * row['time_sec'] / row['duration_sec'])))
    return 1 if row['type'] == 'play_video' else 0

df['score'] = df.apply(calculate_score, axis=1)
df['date'] = pd.to_datetime(df['timestamp']).dt.date

daily_engagement = df.groupby(['date', 'platform'])['score'].sum().reset_index()
print(daily_engagement.head())

engagement_matrix = daily_engagement.pivot(
    index='date',
    columns='platform',
    values='score'
).fillna(0)

plt.figure(figsize=(18, 8))
sns.heatmap(
    engagement_matrix.T,
    cmap='YlOrRd',
    annot=True,
    fmt='.0f',
    linewidths=0.5,
    cbar_kws={'label': 'Engagement Score'}
)
plt.title('Daily User Engagement by Platform', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Platform')
plt.xticks(rotation=45)
plt.tight_layout()

plt.savefig('engagement_heatmap.png', dpi=120)
print("Heatmap saved as 'engagement_heatmap.png'")